In [111]:
import numpy as np
import math
import matplotlib.pyplot as plt
import sympy as sp

In [112]:
def uncertainty_calc(Thrust_Percent):   
    m_dot, P = sp.symbols("m_dot, P")
    
    ve = 3145.7598
    Ae = 0.1527
    
    Me = 4.380
    gamma_c = 1.1173
    gamma_e = 1.213
    #gamma_avg = (gamma_c + gamma_e)/2
    gamma_avg = 1.1414
    Te = 1286.42
    F = 3304.6615
    c_star = 1605
    At = 1796.285e-6
    
    
    
    m_dot_ox_norm = 0.8448
    m_dot_prop_norm = 1.0073
    
    Cd = 0.8
    rho_ox = 1573.2
    A = m_dot_ox_norm/Cd/math.sqrt(2*rho_ox*5.4e5)
    rho_f = 1522
    Dpi = 95.6473e-3
    L = 515.9178e-3
    
    count = 0
    while True:
        if count == 0:
            r = 6.35e-05*(m_dot_ox_norm/(math.pi/4*Dpi**2))**0.5
            m_dot_f = rho_f*math.pi*Dpi*L*r
            m_dot_f_norm = m_dot_f
            OF = m_dot_ox_norm/m_dot_f
            Pe = (1+(gamma_avg-1)/2*Me**2)**(-gamma_avg/(gamma_avg-1))*m_dot*(-2.97619*OF**2 + 10.05952*OF + 1633.18571)/At
            m_dot_prop_update = sp.nsolve(-Thrust_Percent*F + m_dot * ve + Pe * Ae, m_dot, 1.0)
            m_dot_ox_new = abs(m_dot_prop_update - m_dot_f)
            m_dot_ox_old = m_dot_ox_norm
            m_dot_f_old = m_dot_f
            Pe_norm = Pe.subs(m_dot, m_dot_prop_update)
            count += 1
            continue
            
        r = 6.35e-05*(m_dot_ox_new/(math.pi/4*Dpi**2))**0.5
        m_dot_f = rho_f*math.pi*Dpi*L*r
        #m_dot_prop_new = m_dot_ox_new + m_dot_f
        OF = m_dot_ox_new/m_dot_f
        Pe = (1+(gamma_avg-1)/2*Me**2)**(-gamma_avg/(gamma_avg-1))*m_dot*(-2.97619*OF**2 + 10.05952*OF + 1633.18571)/At
        m_dot_prop_update = sp.nsolve(-Thrust_Percent*F + m_dot * ve + Pe * Ae, m_dot, 1.0)
        m_dot_ox_new = abs(m_dot_prop_update - m_dot_f)
        
        if count != 0 and round(abs(m_dot_f - m_dot_f_old),5) == 0 and round(abs(m_dot_ox_new - m_dot_ox_old),5) == 0 and round(abs(m_dot_ox_new + m_dot_f - m_dot_prop_update),5) == 0:
            break
        
        #if count != 0 and round(m_dot_f - m_dot_f_old,10) != 0 and round(m_dot_ox_new - m_dot_ox_old,10) != 0 and round(m_dot_ox_new + m_dot_f - m_dot_prop_update[0],10) != 0:
        else:
            m_dot_ox_old = m_dot_ox_new
            m_dot_f_old = m_dot_f
            count += 1
        
    #print(round(m_dot_ox_new + m_dot_f - m_dot_prop_update[0],10))
    print(f"Number of Iterations = {count}")
    print(f"Oxidizer Mass Flow Rate = {m_dot_ox_new:.3f} kg/s")
    print(f"Fuel Mass Flow Rate Norm = {m_dot_f_norm:.3f} kg/s")
    print(f"Fuel Mass Flow Rate Now = {m_dot_f:.3f} kg/s")
    print(f"Propellant Mass Flow Rate = {m_dot_prop_update:.3f} kg/s")
    print(f"Propellant Mass Flow Uncertainty = {abs(m_dot_prop_norm - m_dot_prop_update):.3f} kg/s")
    print(f"Current O/F = {OF:.3f}")
    print(f"{Thrust_Percent*100:.0f}% Thrust = {Thrust_Percent*F:.3f} N")
    print(f"Current C* = {-2.97619*OF**2 + 10.05952*OF + 1633.18571:.3f} m/s")
    Pe = (1+(gamma_avg-1)/2*Me**2)**(-gamma_avg/(gamma_avg-1))*m_dot_prop_update*(-2.97619*OF**2 + 10.05952*OF + 1633.18571)/At
    print(f"Calculated Thrust = {m_dot_prop_update * ve + Pe * Ae:.3f} N")
    m_dot_ox_change = abs(m_dot_ox_norm - m_dot_ox_new)
    print(f"Oxidizer Mass Flow Rate Uncertainty = {m_dot_ox_change:.3f} kg/s")
    pressure_new = sp.solve(-m_dot_ox_new + Cd * A * sp.sqrt(2*rho_ox*P),P)
    print(f"Pressure drop across the injector = {pressure_new[0]:.3f} Pa")
    pressure_change = abs(5.4e5 - pressure_new[0])/1e5
    print(f"Pressure Rise Uncertainty = {pressure_change:.3f} Bar")
    print(f"Nominal Pe = {Pe_norm:.3f} Pa")
    print(f"Pe Now = {Pe.subs(m_dot, m_dot_prop_update):.3f} Pa")
    
    return A, m_dot_ox_new, m_dot_ox_change, m_dot_f, pressure_new[0], pressure_change, Thrust_Percent
    
[A1, _, ox1, _, Pinj1, delP1, Percent1] = uncertainty_calc(0.90)
print('\n')
[A2, _, ox2, _, Pinj2, delP2, Percent2] = uncertainty_calc(0.95)

#Pe is actually a function of both Pc and O/F, while both Pc and O/F also intertwine in some way, but here Pe is a function of O/F only for simplicity. To be more realistic, a 3D surface equation is needed but the difference is very minimal (like 5 Pa difference in Pe, so the change is actually negligible).
print(f"\nThe following uncertainties are evaluated under the assumption of constant specific heat ratio for the small change in rocket combustion performance, due to the inaccuracy of interpolation needed to obtain the value. But the C* which represents combustion efficiency is considered a variable following the fitted function of C* vs O/F evaluated at a constant 9.0 bar calculated from CEA. This relation of C* is not perfect however, because in actuality C* is also a function of Pc having another associated set of O/F curve, however this value is only used to determine the change in Pe which turns out to be very very minimal, and the slight change in combustion chamber pressure barely affects the performance, therefore the error is negligible, in fact, the whole change in Pe can be neglected.")
print(f"For {Percent1*100:.0f}% Thrust, Mass Flow Rate Uncertainty = {ox1:.3f} kg/s, Pressure Uncertainty = {delP1:.3f} bar")
print(f"For {Percent2*100:.0f}% Thrust, Mass Flow Rate Uncertainty = {ox2:.3f} kg/s, Pressure Uncertainty = {delP2:.3f} bar")
#print(f"\nOxidizer Mass Flow Rate = {0.8 * A * math.sqrt(2 * 1573.2 * Pinj):.3f} kg/s")



Number of Iterations = 16
Oxidizer Mass Flow Rate = 0.753 kg/s
Fuel Mass Flow Rate Norm = 0.162 kg/s
Fuel Mass Flow Rate Now = 0.153 kg/s
Propellant Mass Flow Rate = 0.906 kg/s
Propellant Mass Flow Uncertainty = 0.101 kg/s
Current O/F = 4.910
90% Thrust = 2974.195 N
Current C* = 1610.837 m/s
Calculated Thrust = 2974.195 N
Oxidizer Mass Flow Rate Uncertainty = 0.092 kg/s
Pressure drop across the injector = 429073.080 Pa
Pressure Rise Uncertainty = 1.109 Bar
Nominal Pe = 801.226 Pa
Pe Now = 804.010 Pa


Number of Iterations = 16
Oxidizer Mass Flow Rate = 0.799 kg/s
Fuel Mass Flow Rate Norm = 0.162 kg/s
Fuel Mass Flow Rate Now = 0.158 kg/s
Propellant Mass Flow Rate = 0.957 kg/s
Propellant Mass Flow Uncertainty = 0.050 kg/s
Current O/F = 5.057
95% Thrust = 3139.428 N
Current C* = 1607.952 m/s
Calculated Thrust = 3139.428 N
Oxidizer Mass Flow Rate Uncertainty = 0.046 kg/s
Pressure drop across the injector = 482888.105 Pa
Pressure Rise Uncertainty = 0.571 Bar
Nominal Pe = 845.738 Pa
Pe Now =